## Kansas City vs Austin: Which city is the best for video-game stores?

### Capstone Code

First we import the necessary packages.

In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

# library to handle requests
import requests
# library for data analsysis
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# plotting library
import folium

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.77 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  37.03 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  45.97 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.44 MB/s
vincent-0.4.4- 100% |###################

Next we define the Foursquare client ID and Secret, and Foursquare API version. Note that this information has been hidden after running.

In [2]:
CLIENT_ID = '*****'
CLIENT_SECRET = '*****'
VERSION = '20180604'

First let's analyze Kansas City.

In [13]:
address = 'Kansas City, MO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.100105 -94.5781416


Our search query will be 'video games'. We will define a radius of 10000 meters from the city center and a limit of 50 venues. We add this information to the url, and then save our results as a json file.

In [14]:
search_query = 'Video Games'
radius = 10000
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

We assign the relevant parts of the json file to a variable called **venues**, and then transform it into **dataframe**.

In [15]:
venues = results['response']['venues']

dataframe = json_normalize(venues)

We will keep only the columns that include information to do with location, as well as the venue name. We then define a function that extracts the category of the venue, filter the category for each row, and clean the column names by changing them to only the last term.

In [16]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_mo = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_mo['categories'] = dataframe_mo.apply(get_category_type, axis=1)

dataframe_mo.columns = [column.split('.')[-1] for column in dataframe_mo.columns]

dataframe_mo

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,KC Video Games,Arcade,3909 Broadway Blvd Unit B,US,Kansas City,United States,NaN,4985,"[3909 Broadway Blvd Unit B, Kansas City, MO 64...","[{'label': 'display', 'lat': 39.0563634, 'lng'...",39.056363,-94.590511,64111,MO,58b774ac51d19e7f8b5212b0
1,KC Video Games,Video Game Store,3909 Broadway Blvd,US,Kansas City,United States,NaN,10021,"[3909 Broadway Blvd, Kansas City, MO 64111, Un...","[{'label': 'display', 'lat': 39.05261825014450...",39.052618,-94.676657,64111,MO,54de2628498effd3ec244623
2,"emfluence, llc",Advertising Agency,1720 Wyandotte St,US,Kansas City,United States,at W 18th St,1134,"[1720 Wyandotte St (at W 18th St), Kansas City...","[{'label': 'display', 'lat': 39.09200527085377...",39.092005,-94.586115,64108,MO,52a0d0f211d2e841f63dd513
3,Kansas City,Miscellaneous Shop,NaN,US,Kansas City,United States,NaN,896,"[Kansas City, MO 64106, United States]","[{'label': 'display', 'lat': 39.10553782024686...",39.105538,-94.570484,64106,MO,52e9fb61498eeb50b69c0ed0
4,Video Mania,Video Store,208 Westport Rd,US,Kansas City,United States,NaN,5572,"[208 Westport Rd, Kansas City, MO 64111, Unite...","[{'label': 'display', 'lat': 39.05047596317046...",39.050476,-94.586543,64111,MO,4de06eaec65bcc724f404a39
5,Video Mania,General Entertainment,208 Westport Rd,US,Kansas City,United States,NaN,5223,"[208 Westport Rd, Kansas City, MO 64111, Unite...","[{'label': 'display', 'lat': 39.0539936940061,...",39.053994,-94.589310,64111,MO,4c0ef7dc98102d7ff99ee406
6,Level One Game Shop,Hobby Shop,400 Grand Blvd #4020,US,Kansas City,United States,btwn E 3rd & 5th St,1079,"[400 Grand Blvd #4020 (btwn E 3rd & 5th St), K...","[{'label': 'display', 'lat': 39.1094441, 'lng'...",39.109444,-94.581488,64106,MO,583369e9e753566eaa9d3b41
7,A to Z Video and Photo,Video Store,15 W Linwood Blvd,US,Kansas City,United States,NaN,3631,"[15 W Linwood Blvd, Kansas City, MO 64111, Uni...","[{'label': 'display', 'lat': 39.06810631107306...",39.068106,-94.586310,64111,MO,51a682657dd21ca704e83b35
8,Testimonial Video,Bridal Shop,NaN,US,Kansas City,United States,NaN,4054,"[Kansas City, KS 66101, United States]","[{'label': 'display', 'lat': 39.11426544189453...",39.114265,-94.621384,66101,KS,4fabc062e4b0d040d88e1b9f
9,Video Pros (Brookside Campus),Office,6007 Cherry St,US,Kansas City,United States,61st St,9125,"[6007 Cherry St (61st St), Kansas City, MO 641...","[{'label': 'display', 'lat': 39.01816, 'lng': ...",39.018160,-94.580960,64110,MO,4c87d126334537043e091672


Now we shall generate a map of Kansas City. We generate a map centered around the city center, add a red circle marker to represent the search area, and add the 'Video Games' venues as blue markers.

In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=200,
    color='red',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.1
).add_to(venues_map)

for lat, lng, label in zip(dataframe_mo.lat, dataframe_mo.lng, dataframe_mo.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

Next we make a list of the *IDs* of the venues. We use this to retrieve the rating of each venue, and then find the mean venue rating over Kansas City.

In [18]:
venue_ids_mo = dataframe_mo.id

venue_ratings_mo = []
for i in range(len(venue_ids_mo)):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_ids_mo[i], CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        venue_ratings_mo.append(result['response']['venue']['rating'])
    except:
        np.nan

mean_ratings_mo = np.mean(venue_ratings_mo)
mean_ratings_mo

6.5999999999999996

Now we shall do all of the above for Austin, Texas.

In [19]:
address = 'Austin, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

30.2711286 -97.7436995


In [20]:
search_query = 'Video Games'
radius = 10000
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

venues = results['response']['venues']

dataframe = json_normalize(venues)

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered_tx = dataframe.loc[:, filtered_columns]

dataframe_filtered_tx['categories'] = dataframe_filtered_tx.apply(get_category_type, axis=1)

dataframe_filtered_tx.columns = [column.split('.')[-1] for column in dataframe_filtered_tx.columns]

dataframe_filtered_tx

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,"Resurrected Movies, Video Games, and More",Video Game Store,2025 Guadalupe St #264,US,Austin,United States,Dobie Mall - 2nd Floor,1404,[2025 Guadalupe St #264 (Dobie Mall - 2nd Floo...,"[{'label': 'display', 'lat': 30.2835456849463,...",30.283546,-97.741132,NaN,78705,TX,4bb77d927421a5938e86c040
1,Memory Tree Video Production,Video Store,"Frost Bank Tower, 401 Congress Ave #1540",US,Austin,United States,NaN,461,"[Frost Bank Tower, 401 Congress Ave #1540, Aus...","[{'label': 'display', 'lat': 30.26702268120896...",30.267023,-97.743071,NaN,78701,TX,5784f325cd1041534cfcc425
2,AHD Video,Business Service,209 E 6th St Ste 300D,US,Austin,United States,NaN,475,"[209 E 6th St Ste 300D, Austin, TX 78701, Unit...","[{'label': 'display', 'lat': 30.26744429892386...",30.267444,-97.741199,NaN,78701,TX,54a710b5498e6e2f1e4148ee
3,Texas House Video Control Room,Capitol Building,101-199 W 15th St,US,Austin,United States,NaN,460,"[101-199 W 15th St, Austin, TX 78701, United S...","[{'label': 'display', 'lat': 30.27475214855792...",30.274752,-97.741397,NaN,78701,TX,4d87737c7e8ef04d668747be
4,videotape.co,Music Venue,NaN,US,Austin,United States,NaN,954,"[Austin, TX, United States]","[{'label': 'display', 'lat': 30.266422, 'lng':...",30.266422,-97.735405,NaN,NaN,TX,56e5dac6498e0feda74d6cc0
5,Cuervo Games,General Entertainment,4th,US,Austin,United States,Guadalupe,617,"[4th (Guadalupe), Austin, TX, United States]","[{'label': 'display', 'lat': 30.26684, 'lng': ...",30.266840,-97.747776,NaN,NaN,TX,4cc323b7b2beb1f7b3a0144c
6,Videotape,Music Venue,NaN,US,Austin,United States,NaN,949,"[Austin, TX, United States]","[{'label': 'display', 'lat': 30.26644920618735...",30.266449,-97.735445,NaN,NaN,TX,56da8d00498e53f5fde30328
7,Call4 Legal Video,Tech Startup,301 Congress Ave,US,Austin,United States,NaN,641,"[301 Congress Ave, Austin, TX 78701, United St...","[{'label': 'display', 'lat': 30.26536862499706...",30.265369,-97.743559,NaN,78701,TX,52b36e4d11d20a39698e9fe0
8,Southwest Reporting & Video Service Inc - Austin,Financial or Legal Service,111 Congress Ave Ste 400,US,Austin,United States,NaN,797,"[111 Congress Ave Ste 400, Austin, TX 78701, U...","[{'label': 'display', 'lat': 30.26398, 'lng': ...",30.263980,-97.744290,NaN,78701,TX,54e82df5498e5760d33b8efd
9,I Luv Video,Video Store,4803 Airport Blvd,US,Austin,United States,E 48th 1/2 St,4930,"[4803 Airport Blvd (E 48th 1/2 St), Austin, TX...","[{'label': 'display', 'lat': 30.30808163065903...",30.308082,-97.715430,NaN,78751,TX,4a6bc71ef964a520f0cf1fe3


In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=200,
    color='red',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.1
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered_tx.lat, dataframe_filtered_tx.lng, dataframe_filtered_tx.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [22]:
venue_ids_tx = dataframe_filtered_tx.id

venue_ratings_tx = []
for i in range(len(venue_ids_tx)):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_ids_tx[i], CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        venue_ratings_tx.append(result['response']['venue']['rating'])
    except:
        np.nan

mean_ratings_tx = np.mean(venue_ratings_tx)
mean_ratings_tx

8.0357142857142847

Let's compare these ratings, and then we can solely deal with the dataframe and venues of the winning city.

In [23]:
if (mean_ratings_mo > mean_ratings_tx):
    print('We should go to Kansas City, Missouri!')
    df = dataframe_filtered_mo
else:
    print('We should go to Austin, Texas!')
    df = dataframe_filtered_tx

df.head()

We should go to Austin, Texas!


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,"Resurrected Movies, Video Games, and More",Video Game Store,2025 Guadalupe St #264,US,Austin,United States,Dobie Mall - 2nd Floor,1404,[2025 Guadalupe St #264 (Dobie Mall - 2nd Floo...,"[{'label': 'display', 'lat': 30.2835456849463,...",30.283546,-97.741132,NaN,78705,TX,4bb77d927421a5938e86c040
1,Memory Tree Video Production,Video Store,"Frost Bank Tower, 401 Congress Ave #1540",US,Austin,United States,NaN,461,"[Frost Bank Tower, 401 Congress Ave #1540, Aus...","[{'label': 'display', 'lat': 30.26702268120896...",30.267023,-97.743071,NaN,78701,TX,5784f325cd1041534cfcc425
2,AHD Video,Business Service,209 E 6th St Ste 300D,US,Austin,United States,NaN,475,"[209 E 6th St Ste 300D, Austin, TX 78701, Unit...","[{'label': 'display', 'lat': 30.26744429892386...",30.267444,-97.741199,NaN,78701,TX,54a710b5498e6e2f1e4148ee
3,Texas House Video Control Room,Capitol Building,101-199 W 15th St,US,Austin,United States,NaN,460,"[101-199 W 15th St, Austin, TX 78701, United S...","[{'label': 'display', 'lat': 30.27475214855792...",30.274752,-97.741397,NaN,78701,TX,4d87737c7e8ef04d668747be
4,videotape.co,Music Venue,NaN,US,Austin,United States,NaN,954,"[Austin, TX, United States]","[{'label': 'display', 'lat': 30.266422, 'lng':...",30.266422,-97.735405,NaN,NaN,TX,56e5dac6498e0feda74d6cc0


Let's have a look at all the ratings of the venues of our winning city.

In [24]:
venue_ids = df.id

venue_ratings = []
for i in range(len(venue_ids)):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_ids[i], CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        venue_ratings.append(result['response']['venue']['rating'])
    except:
        venue_ratings.append(np.nan)

venue_ratings

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 8.8,
 nan,
 nan,
 8.2,
 6.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 6.8,
 nan,
 nan,
 nan,
 nan,
 8.9,
 nan,
 nan,
 nan,
 8.1,
 nan,
 7.8,
 nan,
 9.0,
 nan,
 nan,
 nan,
 nan,
 9.3,
 nan,
 8.6,
 nan,
 9.0,
 nan,
 nan,
 7.7,
 6.8,
 7.5,
 nan,
 nan,
 nan]

Now we shall add a category called *ratings* to our dataframe. Then we sort by ratings (highest on top), pick out the columns we are interested in, and drop the venues that don't have a zip code.

In [25]:
df['rating'] = venue_ratings

df = df.sort_values(by=['rating'], ascending=False)

df = df[['name','address','categories','postalCode','rating']]

df = df.dropna(subset = ['postalCode'])

df

,name,address,categories,postalCode,rating
37,Game Over Videogames,3005 S Lamar Blvd Ste B-105B,Video Game Store,78704,9.3
32,Emerald Tavern Games and Cafe,9012 Research Blvd,Gaming Cafe,78758,9.0
41,Precision Camera & Video,2438 W Anderson Ln Ste B4,Camera Store,78757,9.0
24,Tribe Comics and Games,3005 S Lamar Blvd Ste D-113,Comic Shop,78704,8.9
9,I Luv Video,4803 Airport Blvd,Video Store,78751,8.8
39,GameStop,500 E Ben White Blvd Ste 1000,Video Game Store,78704,8.6
12,Vulcan Video South,4411 Russell Dr,Video Store,78745,8.2
28,Whose Turn Is It? Games,2708 S Lamar Blvd #100B,Hobby Shop,78704,8.1
30,Pat's Games,2310 Hancock Dr,Hobby Shop,78756,7.8
44,Adult Video Megaplexxx,7111 S Interstate 35,Adult Boutique,78744,7.7


We can see that some of the categories don't seem correct. Let's manually pick out the categories that seem relevant.

In [26]:
final_df = df.loc[(df.categories == 'Video Game Store') | (df.categories == 'Gaming Cafe') | (df.categories == 'Hobby Shop') | (df.categories == 'Toy / Game Store')]

final_df

,name,address,categories,postalCode,rating
37,Game Over Videogames,3005 S Lamar Blvd Ste B-105B,Video Game Store,78704,9.3
32,Emerald Tavern Games and Cafe,9012 Research Blvd,Gaming Cafe,78758,9.0
39,GameStop,500 E Ben White Blvd Ste 1000,Video Game Store,78704,8.6
28,Whose Turn Is It? Games,2708 S Lamar Blvd #100B,Hobby Shop,78704,8.1
30,Pat's Games,2310 Hancock Dr,Hobby Shop,78756,7.8
46,GameStop,2901 S Capital of Texas Hwy,Video Game Store,78746,7.5
45,GameStop,6001 Airport Blvd,Video Game Store,78752,6.8
0,"Resurrected Movies, Video Games, and More",2025 Guadalupe St #264,Video Game Store,78705,NaN
17,Go! Toys & Games - Barton Creek,2901 S Capital of Texas Hwy,Toy / Game Store,78746,NaN
38,Games Workshop,4302 S Lamar Blvd Ste 600,Toy / Game Store,78704,NaN


Ultimately we are interested in the zip codes, as these will decide where we want to live. So finally we create a list of zip codes, ordered by venue rating.

In [27]:
where_to_live = final_df.postalCode
where_to_live = where_to_live.unique()

print('IMPORTANT ZIP CODES:')
print(where_to_live)

IMPORTANT ZIP CODES:
['78704' '78758' '78756' '78746' '78752' '78705' '78749']


### This concludes our code.